# Verify Keras model from Google Drive

This notebook mounts your Google Drive, loads `keras_model.h5` and `labels.txt` from
`/content/drive/MyDrive/week5-model`, prints the model summary, and runs a demo prediction.

If you uploaded the files to a different Drive folder, update the `model_folder` variable in Cell 2.

In [ ]:
# Cell 1 — ensure required packages and show TensorFlow version
%pip install -q tensorflow h5py pillow
import tensorflow as tf
print('tensorflow', tf.__version__)


In [ ]:
# Cell 2 — mount Google Drive and set model folder (update path if needed)
from google.colab import drive
import os
drive.mount('/content/drive')
model_folder = '/content/drive/MyDrive/week5-model'  # << update if you used a different folder
print('model_folder:', model_folder)
print('exists:', os.path.isdir(model_folder))
if os.path.isdir(model_folder):
    print('files:', os.listdir(model_folder))


In [ ]:
# Cell 3 — load model and labels, print summary
import os
from tensorflow import keras
model_path = os.path.join(model_folder, 'keras_model.h5')
labels_path = os.path.join(model_folder, 'labels.txt')
print('model_path:', model_path)
print('labels_path:', labels_path)
print('model exists:', os.path.exists(model_path))
print('labels exists:', os.path.exists(labels_path))
model = None
try:
    model = keras.models.load_model(model_path, compile=False)
    print('Model loaded. Summary:')
    model.summary()
except Exception as e:
    import traceback
    print('Error loading model:')
    traceback.print_exc()
    model = None
labels = []
if os.path.exists(labels_path):
    with open(labels_path, 'r', encoding='utf-8') as f:
        labels = [l.strip() for l in f if l.strip()]
print('Loaded labels:', labels)


In [ ]:
# Cell 4 — demo prediction: upload an image or use a sample image
from google.colab import files
from PIL import Image
import numpy as np, io, requests, os

if 'model' not in globals() or model is None:
    print('Model not loaded; skipping prediction.')
else:
    print('You may upload an image now (choose Cancel to use the sample image).')
    uploaded = files.upload()
    def preprocess_image(p):
        if isinstance(p, (bytes, io.BytesIO)):
            img = Image.open(io.BytesIO(p)).convert('RGB')
        else:
            img = Image.open(p).convert('RGB')
        img = img.resize((224, 224))
        arr = np.array(img).astype('float32') / 255.0
        return np.expand_dims(arr, 0)
    if uploaded:
        fname = next(iter(uploaded.keys()))
        print('Using uploaded file:', fname)
        x = preprocess_image(uploaded[fname])
    else:
        url = 'https://upload.wikimedia.org/wikipedia/commons/9/99/Happy_smile.jpg'
        print('No upload; fetching sample image from:', url)
        resp = requests.get(url)
        x = preprocess_image(resp.content)
    pred = model.predict(x)
    print('Raw prediction vector:', pred[0].tolist())
    idx = int(np.argmax(pred[0]))
    label_str = labels[idx] if idx < len(labels) else f'index:{idx}'
    print('Predicted index:', idx)
    print('Predicted label:', label_str)
